# Quora Question Pairs (Complete Case Study)

## Introduction 
**Quora** is a place to gain and share knowledge. It's a platform to ask questions and connect with people who contribute unique insights and quality answers. </br>
At the time this competition first presented in Kaggle (5 years ago) the site was getting more than 100 million visitors every single month, now this number increased to 300 million visitors per month, anyone can ask question about the topic he or she want and get answers and Insights from others.


## The Problem from Business Point of View
Because the fact that those huge number of visitors can ask similar questions (which have slightly different words with the same meaning as example), Quora try to identify similar questions (very similar ones which we can say that they are duplicates).<br>
After identifying them they can provide answers to newly asked questions instantly depending on its high similarity to a question already answered on Quora.<br>
This process can improve a lot the User Experience (UX) in the application web or mobile because of three important ideas:<br>
- it will provide answers instantly for the questioner.<br>
- it also provides the best ones which may be upvoted by more people in the old question.<br>
- it helps the user who want to search about a question to quickly find the right answer instead of being overwhelmed by a lot of questions.


## Real World Business Constraints
1.	**No latency concerns** as it is not required to do this process in milliseconds, it may take few seconds or minutes to get the best answer depending on accurate similarity prediction. 
2.	**The cost of mis-classification can be very high**, if someone asks a question and the system recommends answers based on similar question but in fact, they seem similar but they are not so you suggest wrong answers which will affect user experience badly.
3.	You must **define a threshold** for the probability above which you can say they are duplicate.
4.	**Interpretability is partially important** to know what’s your classification depends on?


## Data 
-	CSV file for training data: **train.csv**.
-	Size of the file: 60.4 MB.
-   Every record contains 2 questions IDs and the actual text body for every question and finally the probability of being duplicate. 
-	More info about number of rows, features and sample will be explored below.

## The Problem from Machine Learning Point of View
We can say that it’s a binary classification problem as we need to predict if they are duplicate or not. </br>
Simply we need to solve this function f(q1,q2) = {0,1}
Evaluation Metrics: 
-	**Log-Loss metric** as Quora said in the Kaggle page and its obvious as it’s a probability.
-	**Binary Confusion Matrix** to look for the cost of misclassification as we go on and also the interpretability.
